# 1.1 Implement a decision tree learning algorithm from scratch

In [501]:
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import math

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

load dataset

In [502]:
data = pd.read_csv('magic04.data')
X = data.values[:,:-1] 
Y = data.values[:,-1]

data.head()


,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092,81.8828,g
0,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.261,g
1,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.788,g
2,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.737,g
3,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.462,g
4,51.6240,21.1502,2.9085,0.2420,0.1340,50.8761,43.1887,9.8145,3.6130,238.098,g


Split the dataset for later use

In [503]:
seed = 100                    # Fix random seed for reproducibility
# Shuffle and split the data into train and a 
# concatenation of validation and test sets with a ratio of 0.7/0.3:
X_train, X_val_test, Y_train, Y_val_test = train_test_split(
    X, Y, test_size = 0.3, random_state=seed
)

# Shuffle and split the data into validation and test sets with a ratio of 0.5/0.5:
X_val, X_test, Y_val, Y_test = train_test_split(
    X_val_test, Y_val_test, test_size = 0.5, random_state=seed
)

# Implement a Decision Tree

Data class

In [504]:
class Data:
    def __init__(self, index, mean, majority_label):
        self.index = index
        self.mean = mean
        self.majority_label = majority_label

    def get_index(self):
        return self.index

    def __str__(self):
        return "index: %s, mean: %s, ml: %s" % (self.index, self.mean, self.majority_label)


Node class

In [505]:
class Node:
    def __init__(self, label=None, data=None, left=None, right=None):
        self.label = label
        self.data = data
        self.left = left
        self.right = right

    def get_data(self):
        return self.data

    def is_leaf(self):
        return self.left is None and self.right is None


Tree class

In [506]:
class Tree:
    deleted_nodes = 0
    def __init__(self, dataset=None, label: int = None, root=None): #, branches: List = []):
        """A Tree has 1 root or *is* the root and has arbitrary many branches"""
        self.impurity_measure = None
        self.root = Node()

    # split the datasets based on information-gain and entropy or gini
    def split(self, dataset, split_index):
        split_over, split_under = self.split_over_under(dataset, split_index)

        x_left, y_left = split_over.iloc[:,:-1], split_over.iloc[:,-1]
        x_right, y_right = split_under.iloc[:,:-1], split_under.iloc[:,-1]

        return x_left, y_left, x_right, y_right

    # Splits the data into two dataframes, one containing rows where selected column value is above mean, and one below
    def split_over_under(self, df, column_index, split_method="mean"):
        split_value = 0
        if split_method == "mean":
            split_value = self.get_mean(df, column_index)

        # median is not used, but could be used
        elif split_method == "median":
            split_value = df.iloc[:, column_index].median()
        else:
            raise Exception(f"Split method {split_method} not recognized.")

        x_split_over = df.loc[df.iloc[:,column_index] >= split_value]
        x_split_under = df.loc[df.iloc[:,column_index] < split_value]

        return x_split_over, x_split_under

    # calculates the entropy
    def get_entropy(self, df):
        class_labels = df.iloc[:,-1].unique()
        entropy = 0

        for label in class_labels:
            number_of_rows_current_label = len(df.loc[df.iloc[:,-1] == label])
            p_current_label = number_of_rows_current_label/len(df)
            entropy += -p_current_label * math.log2(p_current_label)
        return entropy

    # returns the sum of the squared possibility that each of the labels are picked (the gini)
    def get_gini(self, df):
        class_labels = df.iloc[:,-1].unique()

        total_gini = 1
        for label in class_labels:
            number_of_rows_current_label = len(df.loc[df.iloc[:,-1] == label])
            p_current_label = number_of_rows_current_label/len(df)
            total_gini -= math.pow(p_current_label, 2)

        return total_gini

    # calculates the information gain
    def get_information_gain(self, df, column_index, split_method="mean"):
        split_over, split_under = self.split_over_under(df, column_index, split_method)

        if self.impurity_measure == "gini":
            impurity_split_over = self.get_gini(split_over)
            impurity_split_under = self.get_gini(split_under)
            prev_impurity_value = self.get_gini(df)
        else:
            impurity_split_over = self.get_entropy(split_over)
            impurity_split_under = self.get_entropy(split_under)
            prev_impurity_value = self.get_entropy(df)

        information_over = (len(split_over)/len(df)) * impurity_split_over
        information_under = (len(split_under)/len(df)) * impurity_split_under
        information = information_over + information_under

        # return information gain (E(parent) - information(child))
        return prev_impurity_value - information

    def predict(self, x):
        return self.predict_tree(x, self.root)

    # predicts a label for a given x
    def predict_tree(self, x, node):
        # if node is a leaf, return the label of the node
        if node.is_leaf():
            return node.label
        # if value at the split index is below split label, traverse down the right side of the tree
        if (x[node.data.index] <= node.data.mean).all():
            return self.predict_tree(x, node.right)
        # continue down the left side
        return self.predict_tree(x, node.left)

    # builds the tree by creating a node and linking it to a left and a right child (recursively)
    def build_tree(self, node, X, y, impurity_measure):
        self.impurity_measure = impurity_measure
        df = pd.concat([X, y], axis=1)

        features, labels = df.iloc[:,:-1], df.iloc[:,-1]
        majority_label = self.get_majority_label(labels)
        if self.has_same_label(labels):
            node.label = majority_label
            return
        elif self.has_identical_features(features):
            node.label = majority_label
            return
        # get optimal split column index
        optimal_split_index = self.get_optimal_split(df)
        # LEFT IS ABOVE AND RIGHT IS BELOW
        x_left, y_left, x_right, y_right = self.split(df, optimal_split_index)

        # get mean used --> save to data
        mean = self.get_mean(df, optimal_split_index)
        node.label = optimal_split_index
        node.data = Data(optimal_split_index, mean, majority_label)

        # create new nodes with values split
        # build the left side of the tree
        if len(y_left) > 0:
            node.left = Node()
            self.build_tree(node.left, x_left, y_left, impurity_measure)
        # build the right side of the tree
        if len(y_right) > 0:
            node.right = Node()
            self.build_tree(node.right, x_right, y_right, impurity_measure)


    def get_optimal_split(self, df):
        optimal_split_index = 0
        # indicates the column with the highest impurity measure based on gini or entropy
        current_information_gain = 0
        num_columns = df.shape[1]
        # get information gain from every possible split and saves the best IG and index
        for index in range(num_columns-1):
            ig = self.get_information_gain(df, index, "mean")

            if ig > current_information_gain:
                current_information_gain, optimal_split_index = ig, index
        return optimal_split_index

    def has_same_label(self, y):
        # if all labels are equal the length of the set will be 1
        if len(set(y)) == 1:
            return True
        return False

    def has_identical_features(self, x):
        # if all features are equal the length of the set will be 1
        if len(set(x)) == 1:
            return True
        return False

    def count_all_labels(self, labels):
        labels_count = {}
        # count the labels and save results in dict
        for label in labels:
            if label in labels_count:
                labels_count[label] += 1
            else:
                labels_count[label] = 1
        return labels_count


    def get_majority_label(self, labels):
        labels_count = self.count_all_labels(labels)
        count = 0
        majority_label = ""
        # get the label with the highest count
        for label in labels_count.keys():
            label_count = labels_count[label]
            if label_count > count:
                majority_label = label
                count = label_count

        return majority_label

    @staticmethod
    def get_mean(dataset, column_index):
        return dataset.iloc[:, column_index].mean()

    def learn(self, X_train, y_train, impurity_measure="entropy", prune=False):
        # split the data in training and pruning 70/30 if prune is true
        if prune:
            X_train, X_prune, y_train, y_prune = train_test_split(
                X_train, y_train, test_size=0.3, random_state=100
            )
            X_prune = pd.DataFrame(X_prune)
            y_prune = pd.DataFrame(y_prune)

        x = pd.DataFrame(X_train)
        y = pd.DataFrame(y_train)

        self.build_tree(self.root,x, y, impurity_measure)
        if prune:
            self.prune_tree(X_prune, y_prune.squeeze(), self.root)
        return self

    def prune_tree(self, X_prune, y_prune, node):
        # return the node accuracy if the node is a leaf (has no children)
        if node.is_leaf():
             return self.count_errors(node.label, y_prune) #self.get_accuracy_score(y_prune, node)

        x_left, y_left, x_right, y_right = self.split(pd.concat([X_prune, y_prune], axis=1), node.data.index)

        # accuracy score of the children
        left_err = self.prune_tree(x_left,y_left, node.left)
        right_err = self.prune_tree(x_right, y_right, node.right)

        if len(y_left) < 1 or len(y_right) < 1 or len(y_prune) < 1 or left_err is None or right_err is None:
            return

        parent_err = self.count_errors(node.data.majority_label, y_prune)

        if parent_err < left_err + right_err:
            self.deleted_nodes += 2
            node.label = node.data.majority_label
            node.left = None
            node.right = None
            return parent_err
        return left_err + right_err


    def count_errors(self, label, y):
        count = self.count_all_labels(y)
        if label in count:
            return len(y) - count[label]
        return len(y)


def print_tree(node, level=0):
    if node != None:
        print_tree(node.left, level + 1)
        print(' ' * 5 * level + '->', node)
        print_tree(node.right, level + 1)


# 1.4 Evaluate Your Algorithm

In [507]:
def get_pred(X, tree):
    y_pred = []
    for x in X:
        y_pred.append(tree.predict(x))
    return np.array(y_pred)

# Run the tree with the specified settings
def run_tree(impurity, prune, x, y):
    print(f"Running Tree with {impurity} and prune set to {prune}...")
    tree = Tree().learn(X_train, Y_train, impurity, prune)
    acc_score = accuracy_score(y, get_pred(x, tree))
    return acc_score

# find the settings that gives the best validation accuracy
def get_best_val_acc(settings):
    val_acc = 0
    current_setting = []
    for setting in settings:
        new_val_acc = run_tree(setting[0], setting[1], X_val, Y_val)
        if new_val_acc > val_acc:
            val_acc = new_val_acc
            current_setting = setting
    return current_setting, val_acc

tree_setting = [["entropy", False], ["entropy", True], ["gini", False], ["gini", True]]

best_setting, val_acc = get_best_val_acc(tree_setting)
print("------------------------------------------------------")
print(f"The best setting for validation accuracy is {best_setting[0]} with pruning set to {best_setting[1]}")
print(f"Validation accuracy = {val_acc}")
print("------------------------------------------------------")
print(f"When using the mentioned settings ({best_setting}) for Test, accuracy = {run_tree(best_setting[0], best_setting[1], X_test, Y_test)}")



Running Tree with entropy and prune set to False...
Running Tree with entropy and prune set to True...
Running Tree with gini and prune set to False...
Running Tree with gini and prune set to True...
------------------------------------------------------
The best setting for validation accuracy is entropy with pruning set to True
Validation accuracy = 0.807570977917981
------------------------------------------------------
Running Tree with entropy and prune set to True...
When using the mentioned settings (['entropy', True]) for Test, accuracy = 0.8233438485804416


# 1.5 Compare to an existing implementation

In [508]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(criterion="entropy")

# Create implemented DecitionTreeClassifier with entropy parameter
clf.fit(X_train, Y_train)

clf_val_acc_entropy = accuracy_score(Y_val, clf.predict(X_val))

# Create implemented DecitionTreeClassifier with gini parameter
clf_gini = tree.DecisionTreeClassifier(criterion="gini")
clf_gini.fit(X_train, Y_train)

clf_val_acc_gini = accuracy_score(Y_val, clf_gini.predict(X_val))

if clf_val_acc_gini > clf_val_acc_entropy:
    best_test_acc = accuracy_score(Y_test, clf_gini.predict(X_test))
    impurity = "gini"
else:
    best_test_acc = accuracy_score(Y_test, clf.predict(X_test))
    impurity = "entropy"
print(f"The best DecitionTreeClassifier accuricy, was with {impurity} as impurity, and the score was {best_test_acc}")




The best DecitionTreeClassifier accuricy, was with entropy as impurity, and the score was 0.8314055380301437
